In [1]:
import pandas as pd
import json
import numpy as np
import os

In [17]:
packets = pd.read_csv(r"C:\Users\lenovo\NETGUARD-AI\data\packets_log.csv")
print(f"[INFO] Loaded {len(packets)} packets")
print(packets.columns.tolist())

with open(r"C:\Users\lenovo\NETGUARD-AI\data\threat_logs.json") as f:
    threats = json.load(f)
print(f"[INFO] Loaded {len(threats)} threat entries")
print(type(threats))
print(threats[:5])

[INFO] Loaded 5062 packets
['Timestamp', 'Src_IP', 'Dst_IP', 'Protocol', 'Src_Port', 'Dst_Port', 'Size']
[INFO] Loaded 2731 threat entries
<class 'list'>
[{'timestamp': '2025-10-24 16:08:06', 'type': 'Possible DoS Attack', 'src': '192.168.29.181', 'dst': '224.0.0.22', 'proto': 2, 'size': 54}, {'timestamp': '2025-10-24 16:08:08', 'type': 'Possible DoS Attack', 'src': '192.168.29.181', 'dst': '13.89.178.26', 'proto': 6, 'size': 172}, {'timestamp': '2025-10-24 16:08:08', 'type': 'Possible DoS Attack', 'src': '192.168.29.181', 'dst': '13.89.178.26', 'proto': 6, 'size': 1494}, {'timestamp': '2025-10-24 16:08:08', 'type': 'Possible DoS Attack', 'src': '192.168.29.181', 'dst': '13.89.178.26', 'proto': 6, 'size': 1494}, {'timestamp': '2025-10-24 16:08:08', 'type': 'Possible DoS Attack', 'src': '192.168.29.181', 'dst': '13.89.178.26', 'proto': 6, 'size': 1494}]


In [18]:
# Convert threat list to dict mapping flow_id -> threat type
threats_dict = {}
for item in threats:
    flow_id = f"{item['src']}_{item['dst']}_{item['proto']}"
    # You can encode threat types as integers (0=no threat, 1=attack)
    threats_dict[flow_id] = 1  # mark all listed threats as 1

In [19]:
packets['flow_id'] = (
    packets['Src_IP'].astype(str) + '_' +
    packets['Dst_IP'].astype(str) + '_' +
    packets['Protocol'].astype(str)
)

packets['threat'] = packets['flow_id'].map(threats_dict).fillna(0).astype(int) # 0 = no threat

In [23]:
agg_funcs = {
    'Size': ['mean', 'max', 'min', 'std'],
    'Timestamp': ['min', 'max', 'count'],
    'threat': 'max'
}

In [24]:
flow_features = packets.groupby('flow_id').agg(agg_funcs)
flow_features.columns = ['_'.join(col).strip() for col in flow_features.columns.values]

In [26]:
packets['flow_id'] = packets['Src_IP'].astype(str) + '_' + packets['Dst_IP'].astype(str) + '_' + packets['Protocol'].astype(str)

packets['threat'] = packets['flow_id'].map(threats_dict).fillna(0).astype(int)


In [28]:
flow_features.reset_index(inplace=True)
flow_features.rename(columns={'threat': 'threat_label'}, inplace=True)

In [32]:
flow_features.drop(columns=['threat_label']).to_csv(r"C:\Users\lenovo\NETGUARD-AI\data\packets_feature.csv", index=False)

flow_features.to_csv(r"C:\Users\lenovo\NETGUARD-AI\data\combined_packets.csv", index=False)